In [ ]:
"""
Purpose: To run spine detection on a generic mesh
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/datasci_tools/datasci_tools/")
sys.path.append("/meshAfterParty/meshAfterParty/")
sys.path.append("/machine_learning_tools/machine_learning_tools/")
sys.path.append("/neuron_morphology_tools/neuron_morphology_tools/")

In [3]:
import datajoint_utils as du
import datajoint as dj

WARNING - 2023-03-31 22:37:59,351 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


Using default Google credentials. There is no ~/.cloudvolume/secrets/google-secret.json set.


WARNING - 2023-03-31 22:38:01,406 - version_utils - You are using microns-morphology-api version 0.0.8, which does not match the latest version on Github, 0.0.11.
WARNING - 2023-03-31 22:38:01,620 - utils - Imported datajoint_plus version, 0.1.0 does not match the latest version on Github, 0.1.6.
INFO - 2023-03-31 22:38:01,626 - settings - Setting enable_python_native_blobs to True
INFO - 2023-03-31 22:38:01,627 - settings - Setting stores to {'h01_skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/skeletons', 'stage': '/mnt/dj-stor01/microns/h01/skeletons'}, 'h01_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/meshes', 'stage': '/mnt/dj-stor01/microns/h01/meshes'}, 'h01_faces': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/faces', 'stage': '/mnt/dj-stor01/microns/h01/faces'}, 'h01_decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/microns/h01/decimated_meshes', 'stage': '/mnt/dj-stor01/microns/h01/decimated_me

Connecting celiib@at-database.ad.bcm.edu:3306


WARNING - 2023-03-31 22:38:02,754 - version_utils - You are using microns-materialization-api version 0.0.9, which does not match the latest version on Github, 0.0.17.
INFO - 2023-03-31 22:38:02,759 - settings - Setting enable_python_native_blobs to True
INFO - 2023-03-31 22:38:09,654 - settings - Setting enable_python_native_blobs to True
ERROR - 2023-03-31 22:38:09,686 - compatibility - Could not add DataJointPlus to ImportMethod.
Traceback (most recent call last):
  File "/datajoint-plus/datajoint_plus/compatibility.py", line 41, in add_datajoint_plus
    obj.parse_hash_info_from_header()
  File "/datajoint-plus/datajoint_plus/base.py", line 354, in parse_hash_info_from_header
    if result[0] == attr:
IndexError: list index out of range
ERROR - 2023-03-31 22:38:09,749 - compatibility - Could not add DataJointPlus to Materialization.
Traceback (most recent call last):
  File "/datajoint-plus/datajoint_plus/compatibility.py", line 41, in add_datajoint_plus
    obj.parse_hash_info_fro

Output()

INFO - 2023-03-31 22:38:19,310 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-03-31 22:38:19,311 - settings - Setting database.user to celiib
INFO - 2023-03-31 22:38:19,311 - settings - Setting database.password to newceliipass
INFO - 2023-03-31 22:38:19,312 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-03-31 22:38:19,313 - settings - Setting database.user to celiib
INFO - 2023-03-31 22:38:19,313 - settings - Setting database.password to newceliipass
INFO - 2023-03-31 22:38:19,314 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2023-03-31 22:38:19,314 - settings - Setting database.user to celiib
INFO - 2023-03-31 22:38:19,314 - settings - Setting database.password to newceliipass
INFO - 2023-03-31 22:38:19,320 - settings - Setting enable_python_native_blobs to True
INFO - 2023-03-31 22:38:20,096 - settings - Setting enable_python_native_blobs to True
INFO - 2023-03-31 22:38:20,181 - settings - Setting database.hos

In [4]:
from dataInterfaceMinnie65 import data_interface as hdju

# Load the data

In [5]:
import trimesh
import numpy as np
import numpy_utils as nu
import trimesh_utils as tu

In [6]:
from pathlib import Path
list(Path("./").iterdir())

[PosixPath('temp'),
 PosixPath('cgal_temp'),
 PosixPath('Harris_Lab_Spine_Identification_vp2_full.ipynb'),
 PosixPath('full_vert.txt'),
 PosixPath('part_vert.txt'),
 PosixPath('full_tri.txt'),
 PosixPath('Harris_Lab_Spine_Identification.ipynb'),
 PosixPath('part_tri.txt'),
 PosixPath('temp_skeleton.cgal'),
 PosixPath('190_mesh.off'),
 PosixPath('.ipynb_checkpoints'),
 PosixPath('190_mesh-cgal_5_0.08.csv'),
 PosixPath('Harris_Lab_Spine_Identification_vp2_full-Copy1.ipynb'),
 PosixPath('mesh_watertight.off'),
 PosixPath('190_mesh-cgal_5_0.08_sdf.csv')]

In [7]:
file_base = "part"
verts_file = f"{file_base}_vert.txt"
tri_file = f"{file_base}_tri.txt"
tri_array = nu.loadtxt(str(tri_file),dtype = "int",delimiter = " ")
verts_array =  nu.loadtxt(str(verts_file),dtype = "float",delimiter = " ")


In [8]:
mesh = tu.mesh_from_vertices_faces(vertices = verts_array, faces = tri_array)
mesh

<trimesh.Trimesh(vertices.shape=(34001, 3), faces.shape=(67451, 3))>

In [9]:
import neuron_visualizations as nviz
nviz.plot_objects(
    mesh,
    buffer = 0
)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning:

Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.



# Decimating

In [10]:
mesh_dec = tu.decimate(mesh,decimation_ratio=0.25,)
mesh_dec = tu.largest_conn_comp(mesh_dec)
nviz.plot_objects(
    mesh_dec,
    buffer=0
)


xvfb-run -n 8523 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Colab_Harris/temp/neuron_94682.off -o /neuron_mesh_tools/Auto_Proofreading/Colab_Harris/temp/neuron_94682_decimated.off -s /neuron_mesh_tools/Auto_Proofreading/Colab_Harris/temp/decimation_meshlab_25178286.mls


# Doing the spine extraction

In [11]:
import spine_utils as spu
import trimesh_utils as tu

In [14]:
# --- shaft parameters ---
spu.soma_vertex_nullification_global = False
spu.skeleton_endpoint_nullification = False

spu.clusters_threshold_global = 6
spu.smoothness_threshold_global = 0.08
spu.shaft_mesh_volume_max_global = 0.4
spu.shaft_close_hole_area_top_2_mean_max_global = 0.4
spu.shaft_mesh_n_faces_min_global = 200

# spine filtering parameters
spu.spine_n_face_threshold_bare_min_global = 310
spu.spine_sk_length_threshold_bare_min_global = 0.6
spu.filter_by_volume_threshold_bare_min_global = 0.011
spu.bbox_oriented_side_max_min_bare_min_global = 0.4
spu.sdf_mean_min_bare_min_global = 0.1
spu.spine_volume_to_spine_area_min_bare_min_global = 0.00002

# head neck segmentation
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400


mesh_dec = tu.decimate(mesh,decimation_ratio=0.25,)
mesh_dec = tu.largest_conn_comp(mesh_dec)

spine_objs = spu.spine_objs_bare_minimum_filt_with_attr_from_branch_obj(
    mesh = mesh_dec,
    plot_filtered_spines = True,
)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

/meshAfterParty/meshAfterParty/spine_utils.py:1636: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

/meshAfterParty/meshAfterParty/spine_utils.py:1637: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

/meshAfterParty/meshAfterParty/spine_utils.py:1667: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

/meshAfterParty/meshAfterParty/spine_utils.

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# Exporting the figure

In [15]:
spu.plot_spines_objs_with_head_neck_and_coordinates(
    spine_objs,
    mesh = mesh_dec,
)

In [16]:
import ipyvolume_utils as ipvu
ipvu.save_to_html(f"{file_base}_spine_detection.html")

# Skeleton Mappings

In [ ]:
import skeleton_utils as sk
#skeletonize_and_clean_connected_branch_CGAL

In [ ]:
skeleton_surface = sk.surface_skeleton(
    mesh_dec,
    plot = True,
    #n_surface_downsampling=3,
)

In [ ]:
nviz.plot_objects(
    skeletons=[skeleton,sk_cgal],
    skeletons_colors=["red","blue"],
    buffer=0
)

In [ ]:
sk_removed

In [ ]:
skeleton = sk.skeleton_cgal(
    mesh_dec,
    plot = True,
    cgal_original_parameters = False,
    max_triangle_angle =1.91986,
    quality_speed_tradeoff=0.1,
    medially_centered_speed_tradeoff=0.2,#0.2,
    area_variation_factor=0.0001,
    max_iterations=500,#500,
    is_medially_centered=True,
    min_edge_length = 1,
    edge_length_multiplier = 0.002,
)

skeleton_no_cycles = sk.remove_cycles_from_skeleton(skeleton,verbose=True,)

sk_cgal,sk_removed = sk.clean_skeleton(
    skeleton_no_cycles,
    min_distance_to_junction = 3,
    return_removed_skeletons = True,
)

nviz.plot_objects(
    mesh_dec,
    skeletons=[sk_cleaned,sk.stack_skeletons(sk_removed)],
    skeletons_colors=["blue","red"],
    buffer=0
)

In [ ]:
# calculating the skeleton endpoints


In [ ]:
spu.spine_objs_with_border_sk_endpoint_and_soma_filter_from_scratch_on_mesh(
    mesh = mesh_dec,
)

In [ ]:
clusters = 6
smoothness = 0.08


# --- determining the shaft ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

sp_objs = spu.spine_objs_with_border_sk_endpoint_and_soma_filter_from_scratch_on_branch_obj(
    mesh = mesh_dec,
    skeleton = sk_cgal,
    clusters = clusters,
    smoothness = smoothness,
    soma_vertex_nullification = False,
    plot_segmentation=True,
    plot_shaft=True,
    plot = True
)

In [ ]:
clusters = 6
smoothness = 0.08


# --- determining the shaft ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

nviz.plot_objects(
    mesh_dec,
    meshes = [k.mesh for k in Out[82]],
    meshes_colors="red",
    buffer = 0
)

In [ ]:
sp_objs_with_att = spu.calculate_spine_attributes_for_list(
        sp_objs,
        calculate_coordinates=False,
)

sp_objs_with_att = [spu.calculate_spine_obj_mesh_skeleton_coordinates(
    spine_obj = k,
    mesh = mesh_dec,
) for k in sp_objs_with_att]

In [ ]:
spine_df = spu.df_from_spine_objs(sp_objs_with_att,add_volume_to_area_ratio=True)

In [ ]:
spine_df.columns

In [ ]:
columns = [
    "skeletal_length",
    "n_faces",
    "volume",
    "sdf_mean",
    "bbox_oriented_side_max",
    "spine_volume_to_spine_area",
]

spine_df[columns]

In [ ]:
not_spines = [30,]
big_spine_would_like_split_by_smoothness = [
    0,
    7,
    9,
    
]
big_spines_with_self_touches = [
    1,
    2,
    4, # self touch and some that could be separated
    5,
]
spine_with_end_on_it = [3]
double_head = [6,11,13]
big_single_head = [8]

In [ ]:
print(spu.spine_n_face_threshold_bare_min_global,
spu.spine_sk_length_threshold_bare_min_global,
spu.filter_by_volume_threshold_bare_min_global,
spu.bbox_oriented_side_max_min_bare_min_global,
spu.sdf_mean_min_bare_min_global,
spu.spine_volume_to_spine_area_min_bare_min_global,)

In [ ]:

# --- shaft parameters ---
spu.shaft_mesh_volume_max_global = 0.7
spu.shaft_close_hole_area_top_2_mean_max_global = 0.6
spu.shaft_mesh_n_faces_min_global = 200

# spine filtering parameters
spu.spine_n_face_threshold_bare_min_global = 310
spu.spine_sk_length_threshold_bare_min_global = 0.6
spu.filter_by_volume_threshold_bare_min_global = 0.011
spu.bbox_oriented_side_max_min_bare_min_global = 0.4
spu.sdf_mean_min_bare_min_global = 0.1
spu.spine_volume_to_spine_area_min_bare_min_global = 0.00002

# head neck segmentation
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400

In [ ]:
verbose = True
sp_objs_filt = spu.filter_spine_objs_by_size_bare_minimum(sp_objs_with_att)
if verbose:
    print(f"AFTER filtering len(sp_objs_filt) = {len(sp_objs_filt)}")

In [ ]:
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 400
sp_objs_filt_head_neck = spu.calculate_spine_attributes_for_list(sp_objs_filt,calculate_head_neck=True, mesh = mesh_dec)

In [ ]:
sp_obj = sp_objs_filt_head_neck[0]
sp_obj.head_mesh,sp_obj.n_heads

In [ ]:
spu.plot_spines_objs_with_head_neck_and_coordinates(
    sp_objs_filt_head_neck,
    mesh = mesh_dec,
)

In [ ]:
sdfs=[0.242122  0.2696905 0.394093  0.439196  0.733061 ], ray_trace_perc = [0.23194992 0.58525092 0.35166066 0.35188387 0.17948695], mesh_sizes = [ 607 4864 2035 1990  111]

In [ ]:
spu.head_smoothness_global

In [ ]:
spu.head_ray_trace_min_global = 0.3
spu.head_face_min_global = 600
spu.head_smoothness_global = 0.03
spu.spine_head_neck(
    sp_objs_filt[21].mesh,
    plot_segmentation = True,
    verbose = True,
    plot_head_neck = True,
    #only_allow_one_connected_component_neck = True
)

# Doing it by hand

In [ ]:
delete_temp_file = True
clusters = 6
smoothness = 0.08
delete_temp_file = True
plot_segmentation = True

current_mesh = mesh_dec

cgal_data,cgal_sdf_data = tu.mesh_segmentation(
    current_mesh,
    #cgal_folder=cgal_folder,
    clusters=clusters,
    smoothness=smoothness,
    return_sdf=True,
    delete_temp_files=delete_temp_file,
    return_meshes=False,
    return_ordered_by_size=False,
    plot_segmentation = plot_segmentation,
)

split_meshes,split_meshes_idx = tu.split_mesh_into_face_groups(current_mesh,cgal_data,return_idx=True,
                                   check_connect_comp = False)

In [ ]:
# import ipyvolume_utils as ipvu
# ipvu.plot_mesh(
#     split_meshes[110]
# )

In [ ]:
meshes = list(split_meshes.values())
functions = [
            "close_hole_area_top_2_mean",
            "n_faces",
            "mesh_volume"
        ]
stats_df = tu.stats_df(meshes,functions)
stats_df

In [ ]:
close_hole_area_top_2_mean_max = 0.6
mesh_volume_max = 0.7
n_faces_min = 200

In [ ]:
stats_df_filt = stats_df.query(f"n_faces > {n_faces_min}")
filt_meshes = [meshes[k] for k in stats_df_filt.index.to_list()]
nviz.plot_objects(
    mesh_dec,
    meshes = filt_meshes,
    meshes_colors="red",
    buffer=0
)

In [ ]:
close_hole_area_top_2_mean_max = spu.shaft_close_hole_area_top_2_mean_max_global
mesh_volume_max = spu.shaft_mesh_volume_max_global
n_faces_min = spu.shaft_mesh_n_faces_min_global

close_hole_area_top_2_mean_max = 1
mesh_volume_max = 1

In [ ]:
query = [
        f"(close_hole_area_top_2_mean > {close_hole_area_top_2_mean_max}) or (mesh_volume > {mesh_volume_max})",
        #f"(close_hole_area_top_2_mean > {close_hole_area_top_2_mean_max}) or (n_faces > {n_faces_max})",
        f"(n_faces > {n_faces_min})"
    ]

import pandas_utils as pu
query_str = pu.query_str(query,table_type="pandas")
stats_df.query(query_str)